In [1]:
##########testing cellchat
library(reticulate)
use_python('/home/jupyter-seton/micromamba/envs/cellchat/bin')

library(anndata)
setwd('/mnt/mpistaff/Cranio_Lab/Louk_Seton/mesenchyme_project_2023/')

anndata_list = c('E8','E9','E10','E11','E12','E13','E14')
annotation_list = c('annotation_combined', 'annotation_combined_fine','annotation_combined_coarse')

for (stage in anndata_list){
    adata_loc = paste0("anndata_objects/adata_",stage,"_combined_annotated.h5ad")
    ad <- read_h5ad(adata_loc)
    for (annotation in annotation_list){
        if(annotation %in% colnames(ad$obs)) { #bit of a nasty if else, but I want to make sure it doesn't hang if the annotation is missing
            counts <- t(as.matrix(ad$layers['original_counts']))
            library.size <- Matrix::colSums(counts)
            data.input <- as(log1p(Matrix::t(Matrix::t(counts)/library.size) * 10000), "dgCMatrix")
            meta <- ad$obs 
            meta$labels <- meta[[annotation]]
            library(CellChat)
            library(patchwork)
            options(stringsAsFactors = FALSE)
            ptm = Sys.time()
            
            cellchat <- createCellChat(object = data.input, meta = meta, group.by = "labels")
            
            CellChatDB <- CellChatDB.mouse #if running on mouse data
            CellChatDB.use <- subsetDB(CellChatDB)
            cellchat@DB <- CellChatDB.use
            
            # subset the expression data of signaling genes for saving computation cost
            cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database
            future::plan("multisession", workers = 16) # do parallel
            cellchat <- identifyOverExpressedGenes(cellchat)
            cellchat <- identifyOverExpressedInteractions(cellchat)
            #> The number of highly variable ligand-receptor pairs used for signaling inference is 692

            execution.time = Sys.time() - ptm
            print(as.numeric(execution.time, units = "secs"))
            #> [1] 13.20763
            # project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
            # cellchat <- projectData(cellchat, PPI.human)
            
            ptm = Sys.time()
            cellchat <- computeCommunProb(cellchat, type = "triMean")
            
            cellchat <- filterCommunication(cellchat, min.cells = 10)
            
            cellchat <- computeCommunProbPathway(cellchat)
            
            cellchat <- aggregateNet(cellchat)
            
            cellchat <- netAnalysis_computeCentrality(cellchat, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways
            
            CellChatDB <- CellChatDB.mouse #if running on mouse data
            CellChatDB.use <- subsetDB(CellChatDB)
            df <- CellChatDB.use$interaction
            df = within(df, { #add pathway annotation
                f <- pathway_name == 'MK' | pathway_name == 'PTN'
                annotation[f] <- 'MK_PTN family'
            }) 

            CellChatDB.use$interaction <- df
            cellchat@DB = CellChatDB.use
            
            saveRDS(cellchat, file = paste0('RDS_objects/cellchat_',stage,'_',annotation,'.rds'))

        } else {
            print(paste0(annotation, ' missing from ', stage))
            }
        }
    }

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: ggplot2



[1] "Create a CellChat object from a data matrix"
The cell barcodes in 'meta' is  AAACGGCTCTGACA.E8_10x21 AATTGATGACCCTC.E8_10x21 ACCTATTGTGCAGT.E8_10x21 ACTGGCCTGGTACT.E8_10x21 ACTGTTACACGGGA.E8_10x21 ACTTGGGACTGCTC.E8_10x21 


Warning message in createCellChat(object = data.input, meta = meta, group.by = "labels"):
“The cell barcodes in 'meta' is different from those in the used data matrix.
              We now simply assign the colnames in the data matrix to the rownames of 'mata'!”
Warning message in createCellChat(object = data.input, meta = meta, group.by = "labels"):
“The 'meta' data does not have a column named `samples`. We now add this column and all cells are assumed to belong to `sample1`!”


Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  Ectoderm - Ectodermal Placode, Ectoderm - Surface Ectoderm, Mesenchyme - Frontonasal Mesenchyme, Mesenchyme - Maxillary Mesenchyme, Mesenchyme - Neural Crest 
The number of highly variable ligand-receptor pairs used for signaling inference is 726 
[1] 73.40322
triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2024-12-04 14:12:38.945596]"
[1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2024-12-04 14:19:51.99552]"


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-2’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "igno

...

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-2’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "igno

[1] "Create a CellChat object from a data matrix"
The cell barcodes in 'meta' is  AAAGGTATCCTCTCGA.14 AAAGTCCGTCAGTCTA.14 AAAGTGAGTAGATCCT.14 AACAAGATCCTGCTAC.14 AACCAACCACCCAATA.14 AACCATGAGACCTTTG.14 


Warning message in createCellChat(object = data.input, meta = meta, group.by = "labels"):
“The cell barcodes in 'meta' is different from those in the used data matrix.
              We now simply assign the colnames in the data matrix to the rownames of 'mata'!”
Warning message in createCellChat(object = data.input, meta = meta, group.by = "labels"):
“The 'meta' data does not have a column named `samples`. We now add this column and all cells are assumed to belong to `sample1`!”


Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  Ectoderm - Committed Neuronal Progenitors, Ectoderm - Fas+ Epithelium, Ectoderm - Immature Neurons, Ectoderm - Mature Neurons, Ectoderm - Neuronal Progenitors, Ectoderm - Non-neuronal Progenitors, Ectoderm - Olfactory Pit Rim - Six1, Ectoderm - Olfactory Pit Rim - Wnt6, Ectoderm - Oral Ectoderm, Ectoderm - Palate Epithelium, Ectoderm - Periderm, Ectoderm - Respiratory Epithelium, Ectoderm - Surface Ectoderm - Dlx2, Ectoderm - Surface Ectoderm - Gjb6, Ectoderm - Vomeronasal Organ, Mesenchyme - Chondrocytes, Mesenchyme - Maxillary Mesenchyme, Mesenchyme - Nasal Cavity Mesenchyme, Mesenchyme - Nostrils, Mesenchyme - Osteoblasts, Mesenchyme - Osteomesenchyme, Mesenchyme - Palatal Shelf, Mesenchyme - Periocular Mesenchyme, Mesenchyme - Philtrum, Mesenchyme - Primary Palate, Mesenchyme - Roof - Sim1, Mesenchyme - Upper Lip, Mesenchyme - VNO Mesenchyme, Mesenchyme - Whisker Pad 
The number of high

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-2’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "igno